In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import torch
from tqdm import tqdm
from OmniEvent.infer import infer, AttrDict, get_model, get_tokenizer

MODEL_PATH = "../models/s2s-mt5-ed/"

In [ ]:
device = torch.device("cpu")
if torch.cuda.is_available():
    device = 'cuda'

def get_model_and_tokenizer(model_name_or_path):
    
    model_args = AttrDict({
        "paradigm": "seq2seq",
        "model_type": "mt5"
    })
    model = get_model(model_args, model_name_or_path)
    model = model.to(device)
    # tokenizer 
    tokenizer = get_tokenizer(model_name_or_path)

    return model, tokenizer

In [ ]:
df = pd.read_csv("../data/data.csv")
df = df.sample(n=50, random_state=42)

In [ ]:
model, tokenizer = get_model_and_tokenizer(MODEL_PATH)

In [ ]:
# from title
for row in df.sample(n=10).itertuples(index=False):
    infer(text=row.Title, task="ED", model=model, tokenizer=tokenizer)

In [ ]:
# get all from articles
ed_results = []
for row in tqdm(df.itertuples(index=False), total=len(df)):
    sentences = [s.strip() for s in row.Content.split(".")]
    ed_article = []
    for sentence in sentences:
        event = infer(text=sentence, task="ED", model=model, tokenizer=tokenizer)[0]
        if len(event['events']) > 0:
            ed_article.append(event)
    ed_results.append(ed_article)

In [ ]:
df['ed_results'] = ed_results

In [ ]:
df.iloc[0]

In [ ]:
df.to_parquet("../data/ed_output.parquet", engine="pyarrow")